In [1]:
!pip install dask
import gzip
import json

In [30]:
from dask.distributed import Client, progress

# Scale up: connect to your own cluster with bmore resources
# see http://dask.pydata.org/en/latest/setup.html
client = Client(processes=False, threads_per_worker=4, n_workers=1, memory_limit='7GB')
client

/opt/conda/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38887 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.17.0.2:38887/status,
Dashboard: http://172.17.0.2:38887/status,Workers: 1
Total threads: 4,Total memory: 6.52 GiB
Status: running,Using processes: False
Comm: inproc://172.17.0.2/4954/10,Workers: 1
Dashboard: http://172.17.0.2:38887/status,Total threads: 4
Started: Just now,Total memory: 6.52 GiB
Comm: inproc://172.17.0.2/4954/13,Total threads: 4
Dashboard: http://172.17.0.2:42725/status,Memory: 6.52 GiB
Nanny: None,


In [31]:
from dask import dataframe as dd

df = dd.read_json('./adzd/AMAZON_FASHION.json.gz')

2023-11-12 11:39:13,061 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.57 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:17,745 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.24 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:18,195 - distributed.worker.memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 4.98 GiB -- Worker memory limit: 6.52 GiB


In [32]:
df.head(10)

2023-11-12 11:39:27,256 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.20 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:29,034 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.26 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:29,451 - distributed.worker.memory - WARNING - Worker is at 76% memory usage. Resuming worker. Process memory: 4.99 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:32,866 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.22 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:33,232 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.22 GiB 

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,5,True,"10 20, 2014",A1D4G1SNUZWQOT,7106116521,Tracy,Exactly what I needed.,perfect replacements!!,1413763200,NaN,<NA>,<NA>
1,2,True,"09 28, 2014",A3DDWDH9PX2YX2,7106116521,Sonja Lau,"I agree with the other review, the opening is ...","I agree with the other review, the opening is ...",1411862400,3.0,<NA>,<NA>
2,4,False,"08 25, 2014",A2MWC41EW7XL15,7106116521,Kathleen,Love these... I am going to order another pack...,My New 'Friends' !!,1408924800,NaN,<NA>,<NA>
3,2,True,"08 24, 2014",A2UH2QQ275NV45,7106116521,Jodi Stoner,too tiny an opening,Two Stars,1408838400,NaN,<NA>,<NA>
4,3,False,"07 27, 2014",A89F3LQADZBS5,7106116521,Alexander D.,Okay,Three Stars,1406419200,NaN,<NA>,<NA>
5,5,True,"07 19, 2014",A29HLOUW0NS0EH,7106116521,Patricia R. Erwin,Exactly what I wanted.,Five Stars,1405728000,NaN,<NA>,<NA>
6,4,True,"05 31, 2014",A7QS961ROI6E0,7106116521,REBECCA S LAYTON,These little plastic backs work great. No mor...,Works great!,1401494400,NaN,<NA>,<NA>
7,3,True,"09 22, 2013",A1BB77SEBQT8VX,B00007GDFV,Darrow H Ankrum II,mother - in - law wanted it as a present for h...,bought as a present,1379808000,NaN,{'Color:': ' Black'},<NA>
8,3,True,"07 17, 2013",AHWOW7D1ABO9C,B00007GDFV,rosieO,"Item is of good quality. Looks great, too. But...",Buxton heiress collection,1374019200,NaN,{'Color:': ' Black'},<NA>
9,3,True,"04 13, 2013",AKS3GULZE0HFC,B00007GDFV,M. Waltman,I had used my last el-cheapo fake leather ciga...,Top Clasp Broke Within 3 days!,1365811200,NaN,{'Color:': ' Black'},<NA>


In [33]:
columns_to_drop = set(df.columns) - set(['reviewText', 'summary', 'overall'])

df = df.drop(columns=columns_to_drop)

In [34]:
meta = { 'overall': 'int', 'reviewText': 'string', 'summary': 'string'}

In [35]:
df['class'] = df['overall'].apply(lambda row: 'positive' if row > 3 else 'negative', meta=('class', 'string'))

2023-11-12 11:39:49,336 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.62 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:39:59,336 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 4.62 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:40:09,361 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [37]:
df.tail()

2023-11-12 11:42:00,336 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.12 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:42:01,560 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.22 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:42:01,635 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 5.23 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:42:10,643 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memo

,overall,reviewText,summary,class
883631,5,I absolutely love this dress!! It's sexy and ...,I absolutely love this dress,positive
883632,5,I'm 5'6 175lbs. I'm on the tall side. I wear a...,I wear a large and ordered a large and it stil...,positive
883633,3,Too big in the chest area!,Three Stars,negative
883634,3,"Too clear in the back, needs lining",Three Stars,negative
883635,5,Ordered and was slightly small. Worked with th...,The quality is excellent and it is so cute,positive


2023-11-12 11:42:31,936 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.40 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:42:41,961 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.40 GiB -- Worker memory limit: 6.52 GiB
2023-11-12 11:42:51,961 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U